In [68]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
import torchvision.models as models
import torch.nn as nn


In [69]:
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import os
from PIL import Image
import tqdm


class CatNotCatDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []
        self.class_to_idx = {'antelope': 0, 'bear': 1, 'bird': 2, 'cat': 3, 'dog': 4}  # Example class mapping

        for class_name, class_idx in self.class_to_idx.items():
            class_path = os.path.join(root_dir, class_name)
            if os.path.isdir(class_path):
                for img_file in os.listdir(class_path):
                    if img_file.lower().endswith(('png', 'jpg', 'jpeg')):
                        img_path = os.path.join(class_path, img_file)
                        self.samples.append((img_path, class_idx))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image, label


In [70]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet normalization
])

dataset = CatNotCatDataset(root_dir='Classified_Animals', transform=transform)

# Splitting dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Creating data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [71]:
# Define the ResNet-18 model class for binary classification
class ResNet18Binary(nn.Module):
    def __init__(self, pretrained=True):
        super(ResNet18Binary, self).__init__()
        self.resnet18 = models.resnet18(pretrained=pretrained)
        num_ftrs = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Linear(num_ftrs, 5)

    def forward(self, x):
        return self.resnet18(x)

In [72]:
import matplotlib.pyplot as plt

def visualize_feature_maps(feature_maps):
    # Assuming feature_maps is a list of tensors from the convolutional layers
    for layer, f_map in enumerate(feature_maps):
        layer += 1  # Start layer indexing at 1
        plt.figure(figsize=(20, 15))
        channels = f_map.shape[1]
        
        for i in range(channels):
            plt.subplot(channels // 8 + 1, 8, i + 1)  # Arrange plots in a grid
            plt.imshow(f_map[0, i].detach().cpu().numpy(), cmap='gray')
            plt.axis('off')
        
        plt.show()


In [73]:
import torch
# Function to train the model
def train_model(model, train_loader, val_loader, num_epochs=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    
    for epoch in tqdm.tqdm(range(num_epochs)):
        model.train()
        running_loss = 0.0
        for images, labels in tqdm.tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        # Print accuracy for the current epoch
        print(f'Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}, ' +
              f'Validation Loss: {val_loss / len(val_loader)}, ' +
              f'Accuracy: {100 * correct / total}%')

    print('Finished Training')

model = ResNet18Binary(pretrained=True)
train_model(model, train_loader, val_loader, num_epochs=5)

 20%|██        | 1/5 [00:19<01:17, 19.37s/it]

Epoch 1, Loss: 1.1155510147412617, Validation Loss: 0.4708970636129379, Accuracy: 93.75%


 40%|████      | 2/5 [00:39<00:59, 19.69s/it]

Epoch 2, Loss: 0.19045050690571466, Validation Loss: 0.22338690608739853, Accuracy: 95.83333333333333%


 60%|██████    | 3/5 [00:58<00:39, 19.69s/it]

Epoch 3, Loss: 0.05179061864813169, Validation Loss: 0.15980256348848343, Accuracy: 95.83333333333333%


 80%|████████  | 4/5 [01:18<00:19, 19.54s/it]

Epoch 4, Loss: 0.03437417916332682, Validation Loss: 0.1376842800527811, Accuracy: 95.83333333333333%


100%|██████████| 5/5 [01:41<00:00, 20.22s/it]

Epoch 5, Loss: 0.012145448941737413, Validation Loss: 0.12844965606927872, Accuracy: 95.83333333333333%
Finished Training
